In [85]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [86]:
global domainName
global rootURL
global alternativeURLs
domainRegex = re.compile(r"https?://?(www\.)?")

### To format any url to http://www. format and eliminate other alternatives

In [87]:
def URLFormatter(url):
    formattedURL = domainRegex.sub('', url).strip().strip('/')
    formattedURL = re.sub("www.", "", formattedURL)
    formattedURL = 'http://www.' + formattedURL
    return formattedURL
# URLFormatter("//domain.com/home")

### Fetching homepage of the website

In [88]:
# Ads http:// to the given URL because it is the only way to check for server response
# If the user will add to the URL directions then they will be deleted
# Example: 'https://moz.com/learn/seo/external-link' will turn to 'https://moz.com/'
# https://stackoverflow.com/questions/32314304/check-if-an-url-is-relative-to-another-ie-they-are-on-the-same-host

rootURL = "https://paws2go.pet/"
domainName = URLFormatter(rootURL)
rootURL = domainName.split('/')[0] + "//" + domainName.split('/')[2]
source_code = requests.get(rootURL)
status_code = source_code.status_code
if(status_code != 200):
    source_code = requests.get(rootURL, headers={"User-Agent": "XY"})
soup = BeautifulSoup(source_code.text, "lxml")
rootTitle = soup.find("title").text
print("fetched "+rootURL+" status code: "+str(status_code))

fetched http://www.paws2go.pet status code: 200


In [89]:
def alternativeURLCreator(url):
    noPrefix = domainRegex.sub('', url).strip().strip('/')
    noPrefix = re.sub("www.", "", noPrefix)
    alternativeURLs = []
    alternativeURLs.append(noPrefix)
    alternativeURLs.append("http://"+noPrefix)
    alternativeURLs.append("http://"+noPrefix+"/")
    alternativeURLs.append("http://"+noPrefix+"//")
    alternativeURLs.append("https://"+noPrefix)
    alternativeURLs.append("https://"+noPrefix+"/")
    alternativeURLs.append("https://"+noPrefix+"//")
    alternativeURLs.append("http://www."+noPrefix)
    alternativeURLs.append("http://www."+noPrefix+"/")
    alternativeURLs.append("http://www."+noPrefix+"//")
    alternativeURLs.append("https://www."+noPrefix)
    alternativeURLs.append("https://www."+noPrefix+"/")
    alternativeURLs.append("https://www."+noPrefix+"//")
    alternativeURLs.append("//"+noPrefix)
    alternativeURLs.append("//"+noPrefix+"/")
    alternativeURLs.append("//"+noPrefix+"//")
    alternativeURLs.append("//www."+noPrefix)
    alternativeURLs.append("//www."+noPrefix+"/")
    alternativeURLs.append("//www."+noPrefix+"//")
    return alternativeURLs

In [90]:
alternativeDomains = alternativeURLCreator(rootURL)
print(alternativeDomains)

['paws2go.pet', 'http://paws2go.pet', 'http://paws2go.pet/', 'http://paws2go.pet//', 'https://paws2go.pet', 'https://paws2go.pet/', 'https://paws2go.pet//', 'http://www.paws2go.pet', 'http://www.paws2go.pet/', 'http://www.paws2go.pet//', 'https://www.paws2go.pet', 'https://www.paws2go.pet/', 'https://www.paws2go.pet//', '//paws2go.pet', '//paws2go.pet/', '//paws2go.pet//', '//www.paws2go.pet', '//www.paws2go.pet/', '//www.paws2go.pet//']


In [91]:
def commonURLChecker(url):
    a = alternativeURLCreator(url)
    b = urlList
    common = list(set(a) & set(b))
    common = str(common).strip("[]").strip('\'')
    return common

In [92]:
externalURLKeywords = ["http", "www", ".com", "https"]
exceptions = ["#", " ", "tel:", "callto:", "mailto:", "wp-content", "wp-login", ".pdf", "twitter.com", "facebook.com", "google.com", "pinterest.com", ".jpg", ".jpeg", ".webm", "javascript:void", "/out/", "?share=", ".atom"]

In [93]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [106]:
# Following function returns the original redirected Url of any link. Meaning if a url "www.hellworld.com/hi" redirects to ""
def urlRedirectChecker(url):
    redirectedURL = BeautifulSoup(requests.get(url).text).link.get("href")
    return redirectedURL

In [115]:
def urlCrawler(url, current_page, max_pages):
    global pageCount
    pageCount += 1
    tempTargetList = []
    outgoing_links = 0
    print("\n\n"+color.BLUE+color.BOLD+"now crawling page: "+color.END+url)
#     Fetching Data
    try:
        source_code = requests.get(url)
    except:
        status_code = source_code.status_code
        print(color.BLUE+color.BOLD+"status code: "+color.END+str(status_code))
    else:
        status_code = source_code.status_code
        print(color.BLUE+color.BOLD+"status code: "+color.END+""+str(status_code))
    if(source_code.status_code != 200):
        source_code = requests.get(rootURL, headers={"User-Agent": "XY"})
        status_code = source_code.status_code
        print(color.BLUE+color.BOLD+"new status code: "+color.END+str(status_code))
#     source_code = requests.get(url)
    if(status_code == 200):
#         not sure whether to keep the above if statement or not. The code works perfectly without it too
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text)
        a = soup.findAll("a")
    #     Assiging Page Data to Dictionary
        if url not in pageDict:
            pageDict[""+url+""] = {}
        data = pageDict[""+url+""]
        try:
            data['title'] = soup.find("title").text
        except:
            print("error: title could not be found")
        data['status_code'] = status_code
        if(url == rootURL): 
            data['clickDepth'] = 0
            data["incoming_links"] = 0
        cur_clickDepth = data['clickDepth']
    #     Run a loop to find target urls
        for eachURL in a:
            href = eachURL.get("href")
            if(eachURL.get("title")):
                anchor = eachURL.get("title")
                anchor = anchor.replace("\n", "").strip()
            else:
                anchor = eachURL.text
                anchor = anchor.replace("\n", "").strip()
            if(href):
                if not any(x in href for x in exceptions):
#                 if any(x in href for x in alternativeURLs) and not any(x in href for x in exceptions):
#                 if not any(x in href for x in alternativeURLs) and not any(x in href for x in exceptions):
    #                 need to change above if statement. Works perfectly for with any x for puppywire.com and with not any x for hotelhalifax.ca but does not work in the alternative case for both
                    if (href == "/" or href in alternativeDomains):
                        tempURL = rootURL
                        tempURL = URLFormatter(tempURL)
                        print("\nfound "+href+" = "+tempURL+" for href in alternativeDomains")
                    elif any(href.startswith(x) for x in alternativeDomains):
                        tempURL = href
                        tempURL = URLFormatter(tempURL)
                        print("\nfound "+href+" = "+tempURL+" for href.startswith(x) for x in alternativeDomains")
                    elif(len(href)>1 and href[0] == "/" and not any(x in href for x in externalURLKeywords)):
                        tempURL = rootURL+""+href
                        tempURL = URLFormatter(tempURL)
                        print("\nfound "+href+" = "+tempURL+" for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords")
                    elif any(x in href for x in externalURLKeywords):
                        tempURL = ""
                        print(color.RED+color.BOLD+"\ndeclined external URL "+color.END+href)
                    else:
#                         added for URLs like "music.html"
                        tempURL = rootURL+"/"+href
                        tempURL = URLFormatter(tempURL)
                        print("\nfound "+href+" = "+tempURL+" for else")
                    if(tempURL!="" and tempURL not in tempTargetList and tempURL != url):
#                     The below function call is making computation 30 times longer
                        tempURL = urlRedirectChecker(tempURL)
                        sourceList.append(url)
                        targetList.append(tempURL)
                        anchorList.append(anchor)
                        hrefList.append(href)
                        print(color.BOLD+"for source: "+color.END+url+color.BOLD+" , appending target: "+color.END+tempURL+color.BOLD+" , anchor: "+color.END+anchor)
                        outgoing_links += 1
                        tempTargetList.append(tempURL)
                        if (tempURL not in urlList):
                            if not any(x in alternativeURLCreator(tempURL) for x in urlList):  
#                       checking if triggered url is a new one and appending ot the urllist if affirmative
                                urlList.append(tempURL)
                                pageDict[""+tempURL+""] = {}                            
                                pageDict[""+tempURL+""]['clickDepth'] = cur_clickDepth+1
                                pageDict[""+tempURL+""]["incoming_links"] = 1
                                print(color.GREEN+color.BOLD+"appending new url: "+color.END+tempURL+color.BOLD+" click depth: "+color.END+str(pageDict[""+tempURL+""]['clickDepth']))
                            else:
                                duplicateURL = commonURLChecker(tempURL)
                                pageDict[""+duplicateURL+""]["incoming_links"] += 1
                                print("URL already in URLlist")
                        else:
                            pageDict[""+tempURL+""]["incoming_links"] += 1
                            print("URL already in URLlist")
                    elif(tempURL!=""):
                        print(color.RED+color.BOLD+"duplicate target link"+color.END)
                else:
                    print(color.RED+color.BOLD+"\ndeclined url (href in exceptions): "+color.END+""+href)
        if(pageCount < max_pages and pageCount < len(urlList)): 
            data['outgoing_links'] = outgoing_links
            urlCrawler(urlList[pageCount], pageCount, max_pages)
        else:
            data['outgoing_links'] = outgoing_links
    else:
#         skeptical about this else statement too. it is meant to add rows for URLs failed to fetch
        if url not in pageDict:
            pageDict[""+url+""] = {}
        data = pageDict[""+url+""]
        data['status_code'] = status_code
        print(color.RED+color.BOLD+"failed to crawl: "+color.END+url+color.BOLD+" status code: "+color.END+status_code)
        if(pageCount < max_pages and pageCount < len(urlList)): 
            data['outgoing_links'] = outgoing_links
            urlCrawler(urlList[pageCount], pageCount, max_pages)
        else:
            data['outgoing_links'] = outgoing_links

In [116]:
pageDict = {}
pageCount = 0

sourceList = []
targetList = []
anchorList = []
hrefList = []

urlList = [rootURL]
outgoing_links = 0
import time
start_time = time.time()
urlCrawler(rootURL, 0, 400)
time = (time.time() - start_time)
print("--- %s seconds ---"%time)



now crawling page: http://www.paws2go.pet
status code: 200

declined url (href in exceptions): #MainContent

found / = http://www.paws2go.pet for href in alternativeDomains
duplicate target link

found /products/dog-doorbell-for-house-training = http://www.paws2go.pet/products/dog-doorbell-for-house-training for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: http://www.paws2go.pet , appending target: https://paws2go.pet/products/dog-doorbell-for-house-training , anchor: BUY
appending new url: https://paws2go.pet/products/dog-doorbell-for-house-training click depth: 1

found /pages/contact-us = http://www.paws2go.pet/pages/contact-us for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: http://www.paws2go.pet , appending target: https://paws2go.pet/pages/contact-us , anchor: CONTACT US
appending new url: https://paws2go.pet/pages/contact-us click depth: 1

found /pages/faqs = http://www.paws2go.pet/

for source: http://www.paws2go.pet , appending target: https://paws2go.pet/products/dog-doorbell-for-house-training , anchor: BUY
URL already in URLlist

found /pages/contact-us = http://www.paws2go.pet/pages/contact-us for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: http://www.paws2go.pet , appending target: https://paws2go.pet/pages/contact-us , anchor: CONTACT US
URL already in URLlist

found /pages/news = http://www.paws2go.pet/pages/news for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: http://www.paws2go.pet , appending target: https://paws2go.pet/pages/news , anchor: PRODUCT REVIEWS
appending new url: https://paws2go.pet/pages/news click depth: 1

found /pages/gallery = http://www.paws2go.pet/pages/gallery for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: http://www.paws2go.pet , appending target: https://paws2go.pet/pages/gallery , anchor: 

for source: https://paws2go.pet/products/dog-doorbell-for-house-training , appending target: https://paws2go.pet/pages/store-policies , anchor: Paws2Go Store Policies
URL already in URLlist

found https://paws2go.pet/pages/faqs = http://www.paws2go.pet/pages/faqs for href.startswith(x) for x in alternativeDomains
for source: https://paws2go.pet/products/dog-doorbell-for-house-training , appending target: https://paws2go.pet/pages/faqs , anchor: Learn More about Paws2Go
URL already in URLlist

declined url (href in exceptions): #

declined url (href in exceptions): //www.facebook.com/sharer.php?u=https://paws2go.pet/products/dog-doorbell-for-house-training

declined url (href in exceptions): //twitter.com/share?text=Wireless%20Dog%20Doorbell&url=https://paws2go.pet/products/dog-doorbell-for-house-training

declined url (href in exceptions): //pinterest.com/pin/create/button/?url=https://paws2go.pet/products/dog-doorbell-for-house-training&media=//cdn.shopify.com/s/files/1/0004/0697/8618

for source: https://paws2go.pet/pages/contact-us , appending target: https://paws2go.pet/pages/faqs , anchor: FAQs
URL already in URLlist

found /pages/about-us = http://www.paws2go.pet/pages/about-us for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/pages/contact-us , appending target: https://paws2go.pet/pages/about-us , anchor: ABOUT US
URL already in URLlist

found /cart = http://www.paws2go.pet/cart for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/pages/contact-us , appending target: https://paws2go.pet/cart , anchor: CartCart
URL already in URLlist

found / = http://www.paws2go.pet for href in alternativeDomains
for source: https://paws2go.pet/pages/contact-us , appending target: https://paws2go.pet/ , anchor: HOME
URL already in URLlist

found /products/dog-doorbell-for-house-training = http://www.paws2go.pet/products/dog-doorbell-for-house-trainin

for source: https://paws2go.pet/pages/faqs , appending target: https://paws2go.pet/pages/training , anchor: Training Hints for Paws2Go
URL already in URLlist

declined url (href in exceptions): https://cdn.shopify.com/s/files/1/0004/0697/8618/files/Paws2Go_Training_Guide.pdf?17

declined url (href in exceptions): https://play.google.com/store/apps/details?id=paws2go.com.paws2go

declined external URL https://itunes.apple.com/us/app/paws2go/id1435506981?mt=8

found https://paws2go.pet/pages/paws2go-app-for-iphones-and-fitbit-smartwatches = http://www.paws2go.pet/pages/paws2go-app-for-iphones-and-fitbit-smartwatches for href.startswith(x) for x in alternativeDomains
for source: https://paws2go.pet/pages/faqs , appending target: https://paws2go.pet/pages/paws2go-app-for-iphones-and-fitbit-smartwatches , anchor: Paws2Go dog doorbell app for iPhones and Fitbit Smartwatches
appending new url: https://paws2go.pet/pages/paws2go-app-for-iphones-and-fitbit-smartwatches click depth: 2

found http

...

for source: https://paws2go.pet/pages/paws2go-app-for-iphones-and-fitbit-smartwatches , appending target: https://paws2go.pet/pages/about-us , anchor: ABOUT US
URL already in URLlist

found /cart = http://www.paws2go.pet/cart for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/pages/paws2go-app-for-iphones-and-fitbit-smartwatches , appending target: https://paws2go.pet/cart , anchor: CartCart
URL already in URLlist

found / = http://www.paws2go.pet for href in alternativeDomains
for source: https://paws2go.pet/pages/paws2go-app-for-iphones-and-fitbit-smartwatches , appending target: https://paws2go.pet/ , anchor: HOME
URL already in URLlist

found /products/dog-doorbell-for-house-training = http://www.paws2go.pet/products/dog-doorbell-for-house-training for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/pages/paws2go-app-for-iphones-and-fitbit-smartwatches , 

for source: https://paws2go.pet/pages/paws2go-travel-mode , appending target: https://paws2go.pet/cart , anchor: CartCart
URL already in URLlist

found / = http://www.paws2go.pet for href in alternativeDomains
for source: https://paws2go.pet/pages/paws2go-travel-mode , appending target: https://paws2go.pet/ , anchor: HOME
URL already in URLlist

found /products/dog-doorbell-for-house-training = http://www.paws2go.pet/products/dog-doorbell-for-house-training for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/pages/paws2go-travel-mode , appending target: https://paws2go.pet/products/dog-doorbell-for-house-training , anchor: BUY
URL already in URLlist

found /pages/contact-us = http://www.paws2go.pet/pages/contact-us for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/pages/paws2go-travel-mode , appending target: https://paws2go.pet/pages/contact-us , anchor: CO

for source: https://paws2go.pet/blogs/news/how-to-use-paws2go-to-potty-train-dog , appending target: https://paws2go.pet/ , anchor: www.paws2go.pet
URL already in URLlist

declined url (href in exceptions): //www.facebook.com/sharer.php?u=https://paws2go.pet/blogs/news/how-to-use-paws2go-to-potty-train-dog

declined url (href in exceptions): //twitter.com/share?text=How%20to%20Use%20Paws2Go%20to%20Potty%20Train%20your%20Dog&url=https://paws2go.pet/blogs/news/how-to-use-paws2go-to-potty-train-dog

declined url (href in exceptions): //pinterest.com/pin/create/button/?url=https://paws2go.pet/blogs/news/how-to-use-paws2go-to-potty-train-dog&media=//cdn.shopify.com/s/files/1/0004/0697/8618/articles/dog-stepping-on-paws2go-dog-doorbell-dark-background_1024x1024.jpg?v=1584244890&description=How%20to%20Use%20Paws2Go%20to%20Potty%20Train%20your%20Dog

found /blogs/news = http://www.paws2go.pet/blogs/news for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for sou

for source: https://paws2go.pet/blogs/news/website-highlights-new-pet-tech-products , appending target: https://paws2go.pet/pages/contact-us , anchor: CONTACT US
URL already in URLlist

found /pages/faqs = http://www.paws2go.pet/pages/faqs for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/blogs/news/website-highlights-new-pet-tech-products , appending target: https://paws2go.pet/pages/faqs , anchor: FAQs
URL already in URLlist

found /pages/about-us = http://www.paws2go.pet/pages/about-us for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/blogs/news/website-highlights-new-pet-tech-products , appending target: https://paws2go.pet/pages/about-us , anchor: ABOUT US
URL already in URLlist

found /cart = http://www.paws2go.pet/cart for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/blogs/news/websi

for source: https://paws2go.pet/blogs/news/dog-training-problem-and-how-to-fix-them , appending target: https://paws2go.pet/pages/about-us , anchor: ABOUT US
URL already in URLlist

found / = http://www.paws2go.pet for href in alternativeDomains
for source: https://paws2go.pet/blogs/news/dog-training-problem-and-how-to-fix-them , appending target: https://paws2go.pet/ , anchor: 
URL already in URLlist

found / = http://www.paws2go.pet for href in alternativeDomains
for source: https://paws2go.pet/blogs/news/dog-training-problem-and-how-to-fix-them , appending target: https://paws2go.pet/ , anchor: HOME
URL already in URLlist

found /products/dog-doorbell-for-house-training = http://www.paws2go.pet/products/dog-doorbell-for-house-training for len(href)>1 and href[0] == / and not any(x in href for x in externalURLKeywords
for source: https://paws2go.pet/blogs/news/dog-training-problem-and-how-to-fix-them , appending target: https://paws2go.pet/products/dog-doorbell-for-house-training , a

In [117]:
pageInfoDF = pd.DataFrame.from_dict({(i): pageDict[i] 
                           for i in pageDict.keys()},
                       orient='index')
pageInfoDF.head(4000)

title  \
http://www.paws2go.pet                              \n    Wireless Dog Doorbell for Potty Training...   
https://paws2go.pet/blogs/news                              \n    Paws2Go Blogs\n    \n    \n    \n     
https://paws2go.pet/blogs/news/dog-training-pro...  \n    Dog Training Problems and How to Fix The...   
https://paws2go.pet/blogs/news/how-to-use-paws2...  \n    How to Use Paws2Go to Potty Train your D...   
https://paws2go.pet/blogs/news/website-highligh...  \n    Website Highlights New Pet Tech Products...   
https://paws2go.pet/cart                            \n    Your Shopping Cart\n    \n    \n    \n  ...   
https://paws2go.pet/pages/about-us                  \n    How We Began | Paws2Go\n    \n    \n    ...   
https://paws2go.pet/pages/contact-us                \n    Contact Us Using Our Support Email or Ph...   
https://paws2go.pet/pages/customer-success-story-1  \n    TOBY'S STORY\n    \n    \n    \n      – ...   
https://paws2go.pet/pages/customer-success-story-2  \n    LUCY'S STORY\n    \n    \n    \n      – ...   
https://paws2go.pet/pages/customer-success-story-3  \n    WINSTON'S STORY\n    \n    \n    \n     ...   
https://paws2go.pet/pages/customer-success-story-4  \n    BUCKEYE'S STORY\n    \n    \n    \n     ...   
https://paws2go.pet/pages/faqs                      \n    Frequently Asked Questions | Paws2Go\n  ...   
https://paws2go.pet/pages/gallery                   \n    Video Gallery | Paws2Go\n    \n    \n   ...   
https://paws2go.pet/pages/mobile-app-privacy-po...  \n    Mobile App Privacy Policy\n    \n    \n ...   
https://paws2go.pet/pages/news                      \n    See the Latest Product Reviews | Paws2Go...   
https://paws2go.pet/pages/paws2go-app-for-iphon...  \n    Paws2Go App for iPhones and Fitbit Smart...   
https://paws2go.pet/pages/paws2go-travel-mode         \n    Paws2Go Travel Mode\n    \n    \n    \n     
https://paws2go.pet/pages/specifications            \n    Technical Specifications | Paws2Go\n    ...   
https://paws2go.pet/pages/store-policies            \n    Store Policies | Paws2Go\n    \n    \n  ...   
https://paws2go.pet/pages/training                  \n    Quick Tips on House Training your Pet wi...   
https://paws2go.pet/products/dog-doorbell-for-h...  \n    Paws2Go Wireless Dog Doorbell - Free Shi...   

                                                    status_code  clickDepth  \
http://www.paws2go.pet                                      200           0   
https://paws2go.pet/blogs/news                              200           1   
https://paws2go.pet/blogs/news/dog-training-pro...          200           2   
https://paws2go.pet/blogs/news/how-to-use-paws2...          200           2   
https://paws2go.pet/blogs/news/website-highligh...          200           2   
https://paws2go.pet/cart                                    200           1   
https://paws2go.pet/pages/about-us                          200           1   
https://paws2go.pet/pages/contact-us                        200           1   
https://paws2go.pet/pages/customer-success-story-1          200           1   
https://paws2go.pet/pages/customer-success-story-2          200           1   
https://paws2go.pet/pages/customer-success-story-3          200           1   
https://paws2go.pet/pages/customer-success-story-4          200           1   
https://paws2go.pet/pages/faqs                              200           1   
https://paws2go.pet/pages/gallery                           200           1   
https://paws2go.pet/pages/mobile-app-privacy-po...          200           1   
https://paws2go.pet/pages/news                              200           1   
https://paws2go.pet/pages/paws2go-app-for-iphon...          200           2   
https://paws2go.pet/pages/paws2go-travel-mode               200           2   
https://paws2go.pet/pages/specifications                    200           1   
https://paws2go.pet/pages/store-policies                    200           1   
https://paws2go.pet/p

In [100]:
Links = {}
Links['source'] = sourceList
Links['target'] = targetList
Links['anchor'] = anchorList
Links['href'] = hrefList
linkInfoDF = pd.DataFrame(Links)
linkInfoDF.head(25000)

source  \
0                               http://www.paws2go.pet   
1                               http://www.paws2go.pet   
2                               http://www.paws2go.pet   
3                               http://www.paws2go.pet   
4                               http://www.paws2go.pet   
5                               http://www.paws2go.pet   
6                               http://www.paws2go.pet   
7                               http://www.paws2go.pet   
8                               http://www.paws2go.pet   
9                               http://www.paws2go.pet   
10                              http://www.paws2go.pet   
11                              http://www.paws2go.pet   
12                              http://www.paws2go.pet   
13                              http://www.paws2go.pet   
14                              http://www.paws2go.pet   
15                              http://www.paws2go.pet   
16   http://www.paws2go.pet/products/dog-doorbell-f...   
17   http://www.paws2go.pet/products/dog-doorbell-f...   
18   http://www.paws2go.pet/products/dog-doorbell-f...   
19   http://www.paws2go.pet/products/dog-doorbell-f...   
20   http://www.paws2go.pet/products/dog-doorbell-f...   
21   http://www.paws2go.pet/products/dog-doorbell-f...   
22   http://www.paws2go.pet/products/dog-doorbell-f...   
23   http://www.paws2go.pet/products/dog-doorbell-f...   
24   http://www.paws2go.pet/products/dog-doorbell-f...   
25   http://www.paws2go.pet/products/dog-doorbell-f...   
26   http://www.paws2go.pet/products/dog-doorbell-f...   
27             http://www.paws2go.pet/pages/contact-us   
28             http://www.paws2go.pet/pages/contact-us   
29             http://www.paws2go.pet/pages/contact-us   
..                                                 ...   
232  http://www.paws2go.pet/blogs/news/how-to-use-p...   
233  http://www.paws2go.pet/blogs/news/how-to-use-p...   
234  http://www.paws2go.pet/blogs/news/how-to-use-p...   
235  http://www.paws2go.pet/blogs/news/how-to-use-p...   
236  http://www.paws2go.pet/blogs/news/how-to-use-p...   
237  http://www.paws2go.pet/blogs/news/how-to-use-p...   
238  http://www.paws2go.pet/blogs/news/website-high...   
239  http://www.paws2go.pet/blogs/news/website-high...   
240  http://www.paws2go.pet/blogs/news/website-high...   
241  http://www.paws2go.pet/blogs/news/website-high...   
242  http://www.paws2go.pet/blogs/news/website-high...   
243  http://www.paws2go.pet/blogs/news/website-high...   
244  http://www.paws2go.pet/blogs/news/website-high...   
245  http://www.paws2go.pet/blogs/news/website-high...   
246  http://www.paws2go.pet/blogs/news/website-high...   
247  http://www.paws2go.pet/blogs/news/website-high...   
248  http://www.paws2go.pet/blogs/news/website-high...   
249  http://www.paws2go.pet/blogs/news/website-high...   
250  http://www.paws2go.pet/blogs/news/dog-training...   
251  http://www.paws2go.pet/blogs/news/dog-training...   
252  http://www.paws2go.pet/blogs/news/dog-training...   
253  http://www.paws2go.pet/blogs/news/dog-training...   
254  http://www.paws2go.pet/blogs/news/dog-training...   
255  http://www.paws2go.pet/blogs/news/dog-training...   
256  http://www.paws2go.pet/blogs/news/dog-training...   
257  http://www.paws2go.pet/blogs/news/dog-training...   
258  http://www.paws2go.pet/blogs/news/dog-training...   
259  http://www.paws2go.pet/blogs/news/dog-training...   
260  http://www.paws2go.pet/blogs/news/dog-training...   
261  http://www.paws2go.pet/blogs/news/dog-training...   

                                                target  \
0    http://www.paws2go.pet/products/dog-doorbell-f...   
1              http://www.paws2go.pet/pages/contact-us   
2                    http://www.paws2go.pet/pages/faqs   
3                http://www.paws2go.pet/pages/about-us   
4                          http://www.paws2go.pet/cart   
5          http://www.paws2go.pet/pages/specifications   
6    http://www.paws2go.pet/pages/customer-su

In [19]:
urlName = rootURL
urlName = urlName.replace("http://","")
urlName = urlName.replace("https://","")
urlName = urlName.replace("www.","")
# urlName = urlName.replace("/","")
linkInfoDF.to_csv(r'C:\Users\hiren\crawly\links_'+urlName+'.csv')
pageInfoDF.to_csv(r'C:\Users\hiren\crawly\pages_'+urlName+'.csv')

In [118]:
urlList

['http://www.paws2go.pet',
 'https://paws2go.pet/products/dog-doorbell-for-house-training',
 'https://paws2go.pet/pages/contact-us',
 'https://paws2go.pet/pages/faqs',
 'https://paws2go.pet/pages/about-us',
 'https://paws2go.pet/cart',
 'https://paws2go.pet/pages/specifications',
 'https://paws2go.pet/pages/customer-success-story-1',
 'https://paws2go.pet/pages/customer-success-story-2',
 'https://paws2go.pet/pages/customer-success-story-3',
 'https://paws2go.pet/pages/customer-success-story-4',
 'https://paws2go.pet/pages/news',
 'https://paws2go.pet/pages/gallery',
 'https://paws2go.pet/pages/training',
 'https://paws2go.pet/blogs/news',
 'https://paws2go.pet/pages/store-policies',
 'https://paws2go.pet/pages/mobile-app-privacy-policy',
 'https://paws2go.pet/pages/paws2go-app-for-iphones-and-fitbit-smartwatches',
 'https://paws2go.pet/pages/paws2go-travel-mode',
 'https://paws2go.pet/blogs/news/how-to-use-paws2go-to-potty-train-dog',
 'https://paws2go.pet/blogs/news/website-highlight

In [119]:
len(urlList)

22

In [22]:
# a = ['a', 'b', 'c']
# b = ['p', 'a', 'r']
# c = "hapay"
# # if any(x in c for x in a) and not any(x in c for x in b):
# #     print("true")
# # else:
# #     print("false")
# if any(x in a for x in b):
#     print(a)
# else:
#     print(b)

['a', 'b', 'c']


In [78]:
mainurl = "https://paws2go.pet/collections"
rootURL = "https://paws2go.pet"
a = BeautifulSoup(requests.get(mainurl).text).findAll("a")
# soup.link.get("href")
for eachURL in a:
    href = eachURL.get("href")
#     print(href)
    if (not any(x in href for x in exceptions) and href != '/'):
        if(len(href)>1 and href[0] == "/" and not any(x in href for x in externalURLKeywords)):
            URL = rootURL+href
            URL = URLFormatter(URL)
#             print(URL)
            print(BeautifulSoup(requests.get(URL).text).link.get("href"))

https://paws2go.pet/products/dog-doorbell-for-house-training
https://paws2go.pet/pages/contact-us
https://paws2go.pet/pages/faqs
https://paws2go.pet/pages/about-us
https://paws2go.pet/products/dog-doorbell-for-house-training
https://paws2go.pet/pages/contact-us
https://paws2go.pet/pages/faqs
https://paws2go.pet/pages/about-us
https://paws2go.pet/cart
https://paws2go.pet/products/dog-doorbell-for-house-training
https://paws2go.pet/pages/contact-us
https://paws2go.pet/pages/news
https://paws2go.pet/pages/gallery
https://paws2go.pet/pages/training
https://paws2go.pet/pages/about-us
https://paws2go.pet/blogs/news
https://paws2go.pet/pages/store-policies
https://paws2go.pet/pages/mobile-app-privacy-policy
